# Introduction



# Dependencies

In [ ]:
!pip3 install pandas==2.0.3 openai==0.28.1 termcolor==2.3.0

# Imports

In [1]:
import json

import pandas as pd
import openai

from termcolor import colored

# Data Handling

https://www.kaggle.com/datasets/yasserh/housing-prices-dataset/

In [56]:
def load_housing_data(preprocess=True):
    furn = ["unfurnished", "semi-furnished", "furnished"]
    hd = pd.read_csv("housing_data.csv")
    if preprocess:
        for col_name in ["mainroad", "guestroom", "basement", "hotwaterheating", "airconditioning", "prefarea"]:
            hd[col_name] = hd[col_name].apply(lambda x: 0 if x == "no" else 1)
        hd["furnishingstatus"] = hd["furnishingstatus"].apply(lambda x: furn.index(x))
    return hd

In [57]:
housing_data = load_housing_data(False)
print(housing_data.dtypes)
housing_data.head()

price                int64
area                 int64
bedrooms             int64
bathrooms            int64
stories              int64
mainroad            object
guestroom           object
basement            object
hotwaterheating     object
airconditioning     object
parking              int64
prefarea            object
furnishingstatus    object
dtype: object


,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [58]:
housing_data = load_housing_data(True)
print(housing_data.dtypes)
housing_data.head()

price               int64
area                int64
bedrooms            int64
bathrooms           int64
stories             int64
mainroad            int64
guestroom           int64
basement            int64
hotwaterheating     int64
airconditioning     int64
parking             int64
prefarea            int64
furnishingstatus    int64
dtype: object


,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,1,0,0,0,1,2,1,2
1,12250000,8960,4,4,4,1,0,0,0,1,3,0,2
2,12250000,9960,3,2,2,1,0,1,0,0,2,1,1
3,12215000,7500,4,2,2,1,0,1,0,1,3,1,2
4,11410000,7420,4,1,2,1,1,1,0,1,2,0,2


# OpenAI

## API Key

In [59]:
try:
    openai.api_key = open("../api_key", "r").read()
    print("API Key loaded")
except FileNotFoundError:
    print("Couldn't find the API key file")

API Key loaded


## Functions

In [ ]:
def filter_lte(housing_data: pd.DataFrame,
               filter_name: str,
               filter_value: int):
    housing_data = housing_data[housing_data[filter_name] <= filter_value]
    return housing_data, len(housing_data)


def filter_gte(housing_data: pd.DataFrame,
               filter_name: str,
               filter_value: int):
    housing_data = housing_data[housing_data[filter_name] >= filter_value]
    return housing_data, len(housing_data)


available_functions = {
    "filter_lte": filter_lte,
    "filter_gte": filter_gte
}

## Function Specifications

In [35]:
functions = [
    {
        "name": "filter_lte",
        "description": "Return the number of houses for which the specified filter is less than or equal to the filter value.",
        "parameters": {
            "type": "object",
            "properties": {
                "filter_name": {
                    "type": "string",
                    "description": "The characteristic to filter by.",
                },
                "filter_value": {
                    "type": "integer",
                    "description": "The value of the characteristic.",
                }
            },
            "required": ["filter_name", "filter_value"],
        }
    },
    {
        "name": "filter_gte",
        "description": "Return the number of houses for which the specified filter is greater than or equal to the filter value.",
        "parameters": {
            "type": "object",
            "properties": {
                "filter_name": {
                    "type": "string",
                    "description": "The characteristic to filter by.",
                },
                "filter_value": {
                    "type": "integer",
                    "description": "The value of the characteristic.",
                }
            },
            "required": ["filter_name", "filter_value"],
        }
    }
]

## API Calls

In [67]:
def call_completion_api(system_content: str, convo: list, model: str = "gpt-3.5-turbo"):

    completion = openai.ChatCompletion.create(
        model=model,
        messages=[{"role": "system", "content": system_content}] + convo,
        functions=functions
    )

    return completion.choices[0]["message"]


def converse(system_content):
    housing_data = load_housing_data()
    system_content = system_content.format(len(housing_data))

    convo = []
    assistant_message = None
    while True:
        if assistant_message is None:
            assistant_message = call_completion_api(system_content, convo)
        convo.append(assistant_message.to_dict_recursive())
        print(colored(assistant_message.content, "blue"))
        
        # Get the user's input
        user_input = input("> ")
        if user_input == "q":
            break

        # Add it to the convo
        convo.append(wrap_user_content(user_input))

        # Get ChatGPT response
        assistant_message = call_completion_api(system_content, convo)
        if assistant_message.get("function_call"):
            convo.append(assistant_message.to_dict_recursive())
            print(colored(assistant_message.function_call, "red"))

            # Extract the function call info
            function_name = assistant_message["function_call"]["name"]
            function_to_call = available_functions[function_name]
            function_args = json.loads(assistant_message["function_call"]["arguments"])

            # Call the function
            housing_data, expected_response = function_to_call(
                housing_data=housing_data,
                filter_name=function_args.get("filter_name"),
                filter_value=function_args.get("filter_value")
            )

            # Add function result to convo
            convo.append(wrap_function_content(function_name, expected_response))
            assistant_message = None
        # else:
        #     convo.append(assistant_message.to_dict_recursive())
        #     print(colored(assistant_message.content, "blue"))
            
    
    return convo


def wrap_user_content(input: str):
    return {"role": "user", "content": input}


def wrap_function_content(function_name: str, function_response):
    return {"role": "function", "name": function_name, "content": str(function_response)}

In [73]:
REALTOR_SYSTEM_CONTENT = \
"""
You are a helpful realtor with a list initially containing {} properties.
Greet the customer and offer to help narrow this selection based on the customer's requirements.
For each property, we have the following characteristics:
    price (asking price in USD),
    area (total floor area in metres squared),
    bedrooms (number of bedrooms),
    bathrooms (number of bathrooms),
    stories (number of stories),
    mainroad (1 if the property is on a main road, otherwise 0),
    guestroom (1 if the property has a guestroom, otherwise 0),
    basement (1 if the property has a basement, otherwise 0),
    hotwaterheating (1 if the property has a water heating system, otherwise 0),
    airconditioning (1 if the property has air conditioning, otherwise 0),
    parking (number of parking spaces),
    prefarea (1 if the property is in a trendy neighbourhood, otherwise 0),
    furnishingstatus (0 if unfurnished, 1 if semi-furnished, 2 if fully furnished)
Help the user to narrow the list.
When there are fewer than 10, advise the user to call the office to arrange viewings.
"""

# I'm looking for a house. The budget is 5000000.

convo = converse(REALTOR_SYSTEM_CONTENT)

Hello! How can I assist you today with your property search?


>  I'm looking for a house. The budget is 5000000.


{
  "name": "filter_lte",
  "arguments": "{\n\"filter_name\": \"price\",\n\"filter_value\": 5000000\n}"
}
Great! Based on your budget of 5000000, there are currently 350 properties available. Let's continue narrowing down the options. 
Could you please provide additional preferences such as the desired area, number of bedrooms, bathrooms, or any specific requirements?


>  do you have any personal feelings on the matter?


As a neutral realtor, my goal is to provide you with accurate information and assist you in finding a property that meets your needs and preferences. I can offer guidance and advice based on market trends and industry knowledge, but ultimately, the decision is yours. Please let me know how I can further assist you in your property search.


>  i want a bungalow


{
  "name": "filter_lte",
  "arguments": "{\n\"filter_name\": \"stories\",\n\"filter_value\": 1\n}"
}
Sure! If you're specifically looking for a bungalow, there are currently 169 properties available. Let's continue narrowing down the options. Do you have any specific preferences for the area, number of bedrooms, bathrooms, or any other requirements?


>  q


In [74]:
for m in convo:
    print(m)

{'role': 'assistant', 'content': 'Hello! How can I assist you today with your property search?'}
{'role': 'user', 'content': "I'm looking for a house. The budget is 5000000."}
{'role': 'assistant', 'content': None, 'function_call': {'name': 'filter_lte', 'arguments': '{\n"filter_name": "price",\n"filter_value": 5000000\n}'}}
{'role': 'function', 'name': 'filter_lte', 'content': '350'}
{'role': 'assistant', 'content': "Great! Based on your budget of 5000000, there are currently 350 properties available. Let's continue narrowing down the options. \nCould you please provide additional preferences such as the desired area, number of bedrooms, bathrooms, or any specific requirements?"}
{'role': 'user', 'content': 'do you have any personal feelings on the matter?'}
{'role': 'assistant', 'content': 'As a neutral realtor, my goal is to provide you with accurate information and assist you in finding a property that meets your needs and preferences. I can offer guidance and advice based on marke